In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.patches as mpatches
import random
import math
import time
import scipy.spatial.distance as distance
from sklearn.covariance import MinCovDet as MCD
import scipy.stats as stats
from numpy import linalg as LA
import progressbar
from pathos.multiprocessing import ProcessingPool as Pool
from sklearn.cluster import KMeans
from sklearn.base import BaseEstimator,TransformerMixin

from sklearn import datasets


In [2]:
class OutlierMahalanobis(TransformerMixin):

    def __init__(self, support_fraction = 0.95, verbose = False, chi2_percentile = 0.995,qqplot=True):
        self.verbose = verbose
        self.support_fraction = support_fraction
        self.chi2 = stats.chi2
        self.mcd = MCD(store_precision = True, support_fraction = support_fraction)
        self.chi2_percentile = chi2_percentile
        self.qqplot=qqplot
        
    def get_params(self):
        return {"support_fraction": self.support_fraction,"chi2_percentile":self.chi2_percentile}

    def set_params(self, **parameters):
        for key,value in parameters.items() :
            setattr(self,key,parameters[key])
        return self

    def fit(self, X,y=None):
        """Prints some summary stats (if verbose is one) and returns the indices of what it consider to be extreme"""
        self.mcd.fit(X)
        d = np.array([distance.mahalanobis(p, self.mcd.location_, self.mcd.precision_ ) for p in X])
        self.d2 = d**2 #MD squared
        n, self.degrees_of_freedom_ = X.shape
        self.iextreme_values = (self.d2 > self.chi2.ppf(self.chi2_percentile, self.degrees_of_freedom_) )
        if self.verbose:
            print("%.3f proportion of outliers at %.3f%% chi2 percentile, "%(self.iextreme_values.sum()/float(n), self.chi2_percentile))
            print("with support fraction %.2f."%self.support_fraction)
            pvalue=stats.kstest(self.d2, lambda x : stats.chi2.cdf(x,df=self.degrees_of_freedom_))[1]
            if pvalue <= 0.01:
                print('Attention : Très forte présomption contre l\'hypothèse nulle p_value : '+str(pvalue))
            elif pvalue <= 0.05:
                print('Attention : Forte présomption contre l\'hypothèse nulle p_value : '+str(pvalue))
            elif pvalue <= 0.1:
                print('Faible présomption contre l\'hypothèse nulle p_value : '+str(pvalue))
            else :
                print('Pas de présomption contre l\'hypothèse nulle. p_value : '+str(pvalue))
            if self.qqplot==True :
                plt.figure(figsize=(10,10))
                stats.probplot(self.d2,dist=stats.chi2(df=self.degrees_of_freedom_), plot=plt)
                plt.title('QQ plot between Mahanalobis distance quantiles and Chi2 quantiles')
                plt.show()

        return self
    
    def transform(self,X):
             
        return X[~self.iextreme_values]

    def plot(self,log=False, sort = False ):
        """
        Cause plotting is always fun.

        log: transform the distance-sq to a log ( distance-sq )
        sort: sort the data according to distnace before plotting
        ifollow: a set if indices to mark with yellow, useful for seeing where data lies across views.

        """
        n = self.d2.shape[0]
        fig = plt.figure(figsize=(10,10))

        x = np.arange( n )
        ax = fig.add_subplot(111)


        transform = (lambda x: x ) if not log else (lambda x: np.log(x))
        chi_line = self.chi2.ppf(self.chi2_percentile, self.degrees_of_freedom_)

        chi_line = transform( chi_line )
        d2 = transform( self.d2 )
        if sort:
            isort = np.argsort( d2 )
            ax.scatter(x, d2[isort], alpha = 0.7, facecolors='none' )
            plt.plot( x, transform(self.chi2.ppf( np.linspace(0,1,n),self.degrees_of_freedom_ )), c="r", label="distribution assuming normal" )


        else:
            ax.scatter(x, d2 )
            extreme_values = d2[ self.iextreme_values ]
            ax.scatter( x[self.iextreme_values], extreme_values, color="r" )

        ax.hlines( chi_line, 0, n,
                        label ="%.1f%% $\chi^2$ quantile"%(100*self.chi2_percentile), linestyles = "dotted" )

        ax.legend()
        ax.set_ylabel("distance squared")
        ax.set_xlabel("observation")
        ax.set_xlim(0, self.d2.shape[0])


        plt.show()
        
    


In [3]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
target_names = iris.target_names
Xreduced=X[:,0:2]
#Xreduced=preprocessin

In [4]:
mh=OutlierMahalanobis(support_fraction = 0.95, chi2_percentile = 0.95)

In [5]:
mh.fit(Xreduced)

In [6]:
class R_pca(TransformerMixin):

    def __init__(self, mu=None, lmbda=None,tol=None,max_iter=1000,iter_print=100,nb_extreme=100):
        self.tol=tol
        self.max_iter=max_iter
        self.iter_print=iter_print
        self.mu=mu
        self.lmbda=lmbda
        self.nb_extreme=nb_extreme
    
    def get_params(self):
        return {"mu": self.mu,"lmbda":self.lmbda,"tol":self.tol,"max_iter":self.max_iter}

    def set_params(self, **parameters):
        for key,value in parameters.items() :
            setattr(self,key,parameters[key])
        return self


    @staticmethod
    def norm_p(M, p):
        return np.sum(np.power(M, p))

    @staticmethod
    def shrink(M, tau):
        return np.sign(M) * np.maximum((np.abs(M) - tau), np.zeros(M.shape))

    def svd_threshold(self, M, tau):
        U, S, V = np.linalg.svd(M, full_matrices=False)
        return np.dot(U, np.dot(np.diag(self.shrink(S, tau)), V))

    def fit(self,X,y=None):
        self.S = np.zeros(X.shape)
        self.Y = np.zeros(X.shape)
        
        if self.mu is None:
            self.mu = np.prod(X.shape) / (4 * self.norm_p(X, 2))

        self.mu_inv = 1 / self.mu

        if self.lmbda is None:
            self.lmbda = 1 / np.sqrt(np.max(X.shape))
        
        
        iter = 0
        err = np.Inf
        Sk = self.S
        Yk = self.Y
        Lk = np.zeros(X.shape)

        if self.tol:
            _tol = self.tol
        else:
            _tol = 1E-7 * self.norm_p(np.abs(X), 2)

        while (err > _tol) and iter < self.max_iter:
            Lk = self.svd_threshold(
                X - Sk + self.mu_inv * Yk, self.mu_inv)
            Sk = self.shrink(
                X - Lk + (self.mu_inv * Yk), self.mu_inv * self.lmbda)
            Yk = Yk + self.mu * (X - Lk - Sk)
            err = self.norm_p(np.abs(X - Lk - Sk), 2)
            iter += 1
            if (iter % self.iter_print) == 0 or iter == 1 or iter > self.max_iter or err <= _tol:
                print('iteration: {0}, error: {1}'.format(iter, err))

        self.L = Lk
        self.S = Sk
        norm=LA.norm(self.S,axis=0)
        self.iextreme_values=np.argsort(norm)[::-1][0:self.nb_extreme]
        
        return self

    def transform(self,X):           
        return X[~self.iextreme_values]

    def plot_normC(self):
        norm=np.sort(LA.norm(self.S,axis=0))[::-1]
        plt.figure(figsize=(10,10))
        plt.plot(norm,'r-')
        plt.legend(['Norme 2 de $C_{0}$'],fontsize=15)
        plt.title('Norme 2 de $C_{0}$ pour chaque point',fontsize=15)
        plt.show()


In [13]:
class R_pca(TransformerMixin):

    def __init__(self, mu=None, lmbda=None,tol=None,max_iter=1000,iter_print=100,nb_extreme=100):
        self.tol=tol
        self.max_iter=max_iter
        self.iter_print=iter_print
        self.mu=mu
        self.lmbda=lmbda
        self.nb_extreme=nb_extreme

    def get_params(self):
        return {"mu": self.mu,"lmbda":self.lmbda,"tol":self.tol,"max_iter":self.max_iter}

    def set_params(self, **parameters):
        for key,value in parameters.items() :
            setattr(self,key,parameters[key])
        return self


    @staticmethod
    def norm_p(M, p):
        return np.sum(np.power(M, p))

    @staticmethod
    def shrink(M, tau):
        return np.sign(M) * np.maximum((np.abs(M) - tau), np.zeros(M.shape))

    def svd_threshold(self, M, tau):
        U, S, V = np.linalg.svd(M, full_matrices=False)
        return np.dot(U, np.dot(np.diag(self.shrink(S, tau)), V))

    def fit(self,X,y=None):
        self.S = np.zeros(X.shape)
        self.Y = np.zeros(X.shape)

        if self.mu is None:
            self.mu = np.prod(X.shape) / (4 * self.norm_p(X, 2))

        self.mu_inv = 1 / self.mu

        if self.lmbda is None:
            self.lmbda = 1 / np.sqrt(np.max(X.shape))


        iter = 0
        err = np.Inf
        Sk = self.S
        Yk = self.Y
        Lk = np.zeros(X.shape)

        if self.tol:
            _tol = self.tol
        else:
            _tol = 1E-7 * self.norm_p(np.abs(X), 2)

        while (err > _tol) and iter < self.max_iter:
            Lk = self.svd_threshold(
                X - Sk + self.mu_inv * Yk, self.mu_inv)
            Sk = self.shrink(
                X - Lk + (self.mu_inv * Yk), self.mu_inv * self.lmbda)
            Yk = Yk + self.mu * (X - Lk - Sk)
            err = self.norm_p(np.abs(X - Lk - Sk), 2)
            iter += 1
            if (iter % self.iter_print) == 0 or iter == 1 or iter > self.max_iter or err <= _tol:
                print('iteration: {0}, error: {1}'.format(iter, err))

        self.L = Lk
        self.S = Sk
        norm=LA.norm(self.S,axis=0)
        self.iextreme_values=np.argsort(norm)[::-1][0:self.nb_extreme]

        return self

    def transform(self,X):
        return X[~self.iextreme_values]

    def plot_normC(self):
        norm=np.sort(LA.norm(self.S,axis=0))[::-1]
        plt.figure(figsize=(10,10))
        plt.plot(norm,'r-')
        plt.legend(['Norme 2 de $C_{0}$'],fontsize=15)
        plt.title('Norme 2 de $C_{0}$ pour chaque point',fontsize=15)
        plt.show()


In [14]:
rpca=R_pca()

In [15]:
rpca.fit(np.array(pd.DataFrame(Xreduced).transpose()))

iteration: 1, error: 6604.928315828149
iteration: 60, error: 0.00022880075027244633


In [19]:
np.array(pd.DataFrame(Xreduced).transpose())

array([[ 5.1,  4.9,  4.7,  4.6,  5. ,  5.4,  4.6,  5. ,  4.4,  4.9,  5.4,
         4.8,  4.8,  4.3,  5.8,  5.7,  5.4,  5.1,  5.7,  5.1,  5.4,  5.1,
         4.6,  5.1,  4.8,  5. ,  5. ,  5.2,  5.2,  4.7,  4.8,  5.4,  5.2,
         5.5,  4.9,  5. ,  5.5,  4.9,  4.4,  5.1,  5. ,  4.5,  4.4,  5. ,
         5.1,  4.8,  5.1,  4.6,  5.3,  5. ,  7. ,  6.4,  6.9,  5.5,  6.5,
         5.7,  6.3,  4.9,  6.6,  5.2,  5. ,  5.9,  6. ,  6.1,  5.6,  6.7,
         5.6,  5.8,  6.2,  5.6,  5.9,  6.1,  6.3,  6.1,  6.4,  6.6,  6.8,
         6.7,  6. ,  5.7,  5.5,  5.5,  5.8,  6. ,  5.4,  6. ,  6.7,  6.3,
         5.6,  5.5,  5.5,  6.1,  5.8,  5. ,  5.6,  5.7,  5.7,  6.2,  5.1,
         5.7,  6.3,  5.8,  7.1,  6.3,  6.5,  7.6,  4.9,  7.3,  6.7,  7.2,
         6.5,  6.4,  6.8,  5.7,  5.8,  6.4,  6.5,  7.7,  7.7,  6. ,  6.9,
         5.6,  7.7,  6.3,  6.7,  7.2,  6.2,  6.1,  6.4,  7.2,  7.4,  7.9,
         6.4,  6.3,  6.1,  7.7,  6.3,  6.4,  6. ,  6.9,  6.7,  6.9,  5.8,
         6.8,  6.7,  6.7,  6.3,  6.5, 

In [20]:
rpca.iextreme_values

array([118, 122, 130, 135, 105,  68, 108,  87, 107, 119,  62, 129, 146,
        72,  76, 102, 111, 125,  54, 123, 112, 134,  58, 132, 128, 139,
        52, 141, 145,  77,  60,  53,  50, 133,  75,  74, 120, 131, 113,
       126, 140,  65,  86,  80,  81,  83, 147, 104, 116,  92, 103, 143,
        69,  71,  73,  79,  97, 117,  89, 142,  67, 101,  82,  63, 137,
       124, 144, 109,  93,  90,  78, 110, 114,  94, 127,  91,  15,  32,
        51, 115,  55,  99,  33, 138,  22,  98, 121,  57, 149,  61,  96,
        19,  44,  46, 100,  56,   5,  16,  64,  21], dtype=int64)

In [22]:
rpca.transform(np.array(pd.DataFrame(Xreduced)))

array([[ 5.4,  3.4],
       [ 5.2,  3.5],
       [ 5.1,  3.8],
       [ 5.8,  4. ],
       [ 5.1,  3.8],
       [ 5.5,  2.4],
       [ 4.5,  2.3],
       [ 6. ,  2.2],
       [ 4.4,  3.2],
       [ 4.8,  3.1],
       [ 6.3,  2.3],
       [ 5.4,  3.4],
       [ 4.6,  3.1],
       [ 6.7,  3. ],
       [ 6.1,  2.8],
       [ 4.6,  3.2],
       [ 4.4,  3. ],
       [ 4.8,  3.4],
       [ 5.7,  3. ],
       [ 5. ,  3.4],
       [ 4.9,  3.1],
       [ 5.7,  4.4],
       [ 6.1,  3. ],
       [ 5.1,  3.5],
       [ 5.1,  3.7],
       [ 5.4,  3.7],
       [ 6.2,  2.9],
       [ 4.4,  2.9],
       [ 5. ,  3.6],
       [ 6.3,  2.5],
       [ 5.5,  2.5],
       [ 5.7,  2.9],
       [ 5.7,  2.8],
       [ 5.4,  3.9],
       [ 6.4,  2.9],
       [ 6.6,  3. ],
       [ 4.7,  3.2],
       [ 5.7,  3.8],
       [ 5.5,  3.5],
       [ 5.1,  3.3],
       [ 4.9,  3.1],
       [ 5.4,  3. ],
       [ 6.1,  2.9],
       [ 5.6,  2.5],
       [ 6.2,  2.2],
       [ 5.6,  3. ],
       [ 4.7,  3.2],
       [ 4.8,

In [89]:
rpca.iextreme_values

array([0, 1], dtype=int64)

In [90]:
class OutliersKmeans(TransformerMixin):
    ''' v.0.2 OutliersKmeans : Find outliers using Kmeans. Only for semi-supervised outlier detection'''
    def __init__(self,kmeans,parallel=False,showbar=False):
        self.kmeans=kmeans
        self.parallel=parallel
        self.showbar=showbar
        
    def get_params(self):
        return {"kmeans": self.kmeans}

    def set_params(self, **parameters):
        for key,value in parameters.items() :
            setattr(self,key,parameters[key])
        return self  
    
    
    def fit(self,X,y=None):
        
        self.kmeans.fit(X)
        preds=self.kmeans.predict(X)
        self.centers=self.kmeans.cluster_centers_
        self.d=self.find_extreme_point(self.centers,preds,X)
        self.iextreme_values=[]
        if self.showbar == True :
            widgets = [progressbar.Percentage(), progressbar.Bar()]
            bar = progressbar.ProgressBar(widgets=widgets, max_value=len(points)).start()
            j=0
        if self.parallel==True:
            p=Pool(4)
            g=lambda point:not(self.in_any_circle(point,self.centers,self.d))
            self.iextreme_values=p.map(g, X)

        else:
            for point in X:

                self.iextreme_values.append(not(self.in_any_circle(point,self.centers,self.d)))

                if self.showbar== True :
                    self.sleep(0.1)
                    bar.update(j)
                    j=j+1

        if self.showbar== True :
            bar.finish()
        
        self.iextreme_values=np.array(self.iextreme_values)
        return self
    
    def transform(self,X):
           
        return X[~self.iextreme_values]
        
        
    def sleep(self,delay):
        time.sleep(delay)


    def to_center_distances(self,X,mu):
        D=[]
        for i in range(len(X)):
            D.append(np.sum((X[i]-mu)**2))
        return D

    def find_extreme_point(self,centers,ypred,X):
        K=len(centers)
        d={}
        for i in range(K):
            X_in_cluster=X[np.where(ypred==i),:][0]
            D=self.to_center_distances(X_in_cluster,centers[i])
            indexmax=np.argmax(D)
            d[i+1]=(X[indexmax],math.sqrt(np.max(D)))
        return d

    def draw_circle(self,Npoints,radius,center):
        circle_slice=2*math.pi/Npoints
        p=[]
        for i in range(Npoints):
            angle=circle_slice*i
            newx=center[0]+radius*math.cos(angle)
            newy=center[1]+radius*math.sin(angle)
            p.append([newx,newy])
        return np.array(p)

    def bounds(self,centers,ypred,Xreduced,Npoints=100):
        d=self.find_extreme_point(centers,ypred,Xreduced)
        K=len(d)
        circles=[]
        for i in range(K):
            center=centers[i]
            radius=d[i+1][1]
            circles.append(self.draw_circle(Npoints,radius,center))
        return circles

    def in_circle(self,point,center,radius):
        return math.sqrt(np.sum((point-center)**2))<=radius

    def in_any_circle(self,point,centers,d):
        K=len(d)
        incirclek=[]
        for k in range(K):
            center=centers[k]
            radius=d[k+1][1]
            incirclek.append(self.in_circle(point,center,radius))
        return np.any(incirclek)


In [91]:
kmeans = KMeans(init='k-means++', n_clusters=5, n_init=10,random_state=5)

In [92]:
outk=OutliersKmeans(kmeans=kmeans)

In [93]:
outk.fit(Xreduced)

In [94]:
outliers=[]
outliers.append([5,2.5])
outliers.append([8,5])
outliers.append([4,1.5])
outliers.append([6,3])
outliers.append([7,1.5])

In [96]:
outk.transform(Xreduced)

array([[ 5.1,  3.5],
       [ 4.9,  3. ],
       [ 4.7,  3.2],
       [ 4.6,  3.1],
       [ 5. ,  3.6],
       [ 5.4,  3.9],
       [ 4.6,  3.4],
       [ 5. ,  3.4],
       [ 4.4,  2.9],
       [ 4.9,  3.1],
       [ 5.4,  3.7],
       [ 4.8,  3.4],
       [ 4.8,  3. ],
       [ 4.3,  3. ],
       [ 5.8,  4. ],
       [ 5.7,  4.4],
       [ 5.4,  3.9],
       [ 5.1,  3.5],
       [ 5.7,  3.8],
       [ 5.1,  3.8],
       [ 5.4,  3.4],
       [ 5.1,  3.7],
       [ 4.6,  3.6],
       [ 5.1,  3.3],
       [ 4.8,  3.4],
       [ 5. ,  3. ],
       [ 5. ,  3.4],
       [ 5.2,  3.5],
       [ 5.2,  3.4],
       [ 4.7,  3.2],
       [ 4.8,  3.1],
       [ 5.4,  3.4],
       [ 5.2,  4.1],
       [ 5.5,  4.2],
       [ 4.9,  3.1],
       [ 5. ,  3.2],
       [ 5.5,  3.5],
       [ 4.9,  3.1],
       [ 4.4,  3. ],
       [ 5.1,  3.4],
       [ 5. ,  3.5],
       [ 4.5,  2.3],
       [ 4.4,  3.2],
       [ 5. ,  3.5],
       [ 5.1,  3.8],
       [ 4.8,  3. ],
       [ 5.1,  3.8],
       [ 4.6,